In [1]:
#Instalar estas librerías en caso de no tenerlas

#pip install pandas
#pip install numpy
#pip install scikit-learn
#pip install imbalanced-learn
#pip install matplotlib
#pip install seaborn
#pip install plotly

In [2]:
#Abrimos la base de datos de Clientes
#Importamos la librería pandas y matplotlib 
import pandas as pd
import numpy as np
%matplotlib widget
pd.options.display.max_columns = None
#Llamamos la tabla cliente.csv
df= pd.read_csv("reto.csv")
#Mostramos la información de la tabla (columnas, valores no nulos y tipo de dato)
df.info()
import warnings
warnings.filterwarnings('ignore')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [3]:
#Llamamos las librerías que usaremos

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics as mt
from IPython.display import Image as PImage
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [4]:
#Visualizamos la data
df.head(-30)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362585,741,CASH_OUT,0.00,C312737633,0.00,0.00,C1400061387,267522.87,267522.87,1,0
6362586,741,TRANSFER,62297.18,C1565612524,62297.18,0.00,C435734396,0.00,0.00,1,0
6362587,741,CASH_OUT,62297.18,C1597957921,62297.18,0.00,C734451013,1060553.11,1122850.29,1,0
6362588,741,TRANSFER,114149.44,C1157446475,114149.44,0.00,C1367473638,0.00,0.00,1,0


# Preparación de base para modelo

In [5]:
#Verificamos si hay valores nulos
df.isnull().values.any()

False

In [6]:
#Verificamos si hay valores duplicados
df.duplicated().any()

False

In [7]:
#Se eliminan las variables categóricas y se agregan variables dummy de la variable type para analizar por tipo de transacción
df2 = pd.concat([df.drop(columns=['isFraud','isFlaggedFraud']),pd.get_dummies(df.type)] ,axis=1)
df2 = df2.drop(['type', 'nameOrig', 'nameDest'],1)
df2 = pd.concat([df2,df.iloc[:,-2:]],axis=1)
pd.DataFrame.head(df2)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 13 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   amount          float64
 2   oldbalanceOrg   float64
 3   newbalanceOrig  float64
 4   oldbalanceDest  float64
 5   newbalanceDest  float64
 6   CASH_IN         uint8  
 7   CASH_OUT        uint8  
 8   DEBIT           uint8  
 9   PAYMENT         uint8  
 10  TRANSFER        uint8  
 11  isFraud         int64  
 12  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), uint8(5)
memory usage: 418.7 MB


Contamos cuantas datos hay de las dos variables que presentan transacciones fraudulentas

In [8]:
#Columnas que se utilizarán para entrenar el modelo
cols = ["amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest", 'CASH_OUT','TRANSFER']

# Haciendo predicciones

In [10]:
import joblib

# Cargar el modelo desde el archivo
loaded_model = joblib.load('modelo_entrenado.pkl')

In [11]:
X=df2[cols]
y =df2[['isFraud']]

In [12]:
X_orig = X.copy()
y_orig = y.copy()

In [13]:
# Making predictions
y_pred_dt = loaded_model.predict(X)

# Printing classification report
print(classification_report(y, y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96   6354407
           1       0.02      1.00      0.03      8213

    accuracy                           0.92   6362620
   macro avg       0.51      0.96      0.50   6362620
weighted avg       1.00      0.92      0.96   6362620



In [15]:
# Printing confusion matrix and accuracy
print(confusion_matrix(y,y_pred_dt))
print('Accuracy of Decision Tree :',accuracy_score(y,y_pred_dt))
print('Sensitivity of Decision Tree :',recall_score(y,y_pred_dt))

[[5857443  496964]
 [     30    8183]]
Accuracy of Decision Tree : 0.9218884673294964
Sensitivity of Decision Tree : 0.9963472543528552


El modelo predice con certeza un 92.18% de casos de fraudes, considerando una base de datos con más de 6 millones de datos y obteniendo casi 6 millones de resultados correctos, se considera aceptable el modelo. Cabe destacar que se ajustaron los hiperparámetros para evitar un sobreajuste, por lo cual la precisión de la predicción también puede mejorar.

In [16]:
kfold_validation=KFold(10)

results=cross_val_score(loaded_model,X,y,cv=kfold_validation)
print(results)
print(np.mean(results))

[0.99957407 0.99933204 0.99978468 0.99971867 0.99975796 0.99965895
 0.99968409 0.99970295 0.9997501  0.99790652]
0.999487003781461


# Almacenar los resultados

In [24]:
df_final=df
# Agregar las predicciones como una nueva columna en el DataFrame 'y'
df_final['Predicciones'] = y_pred_dt

# Guardar en un archivo CSV
df_final.to_csv('resultados_predicciones.csv', index=False)